In [1]:
import pandas as pd
import os
import numpy as np
from Enrichment import *
from collections import Counter
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt 

In [2]:
# set the correct working directory
if os.path.isdir('M:/'):
    os.chdir('M:\Box\Jake-Jegga\IPF Drug Discovery\Results')
elif os.path.isdir('E:/'):
    os.chdir('E:/Box Sync/Jake-Jegga/IPF Drug Discovery/Results')
else:
    os.chdir('/Users/wano3m/Box Sync/Jake-Jegga/IPF Drug Discovery/Results')
top_candidates = pd.read_excel('91+nintedabin enrichment connectivity report.xlsx',sheet_name=0,index_col=0)
top_candidates.set_index(top_candidates.index + '_' + top_candidates.Cell+'_'+top_candidates.Dose,inplace=True)
enrichment_profiles = pd.read_table('91+nintedanib enrichment by dose by cell type report.txt',index_col = 0)
enrichment_profiles.set_index(enrichment_profiles.index + '_' + enrichment_profiles.cellline,inplace=True)
enrich = SEA('../../../Ontology_Info/')

In [3]:
# calculate IPF enrichment
ipf_genes = pd.read_table('IPF DEG combinded logFC.gct',skiprows=2,index_col=0)
ipf_up = ipf_genes.columns[ipf_genes.iloc[0,:].astype(float)>=0.6].tolist()
ipf_dn = ipf_genes.columns[ipf_genes.iloc[0,:].astype(float)<=-0.6].tolist()
ipf_enrichment = pd.Series()
for gl in [ipf_up,ipf_dn]:
#     for lib in ['BP',' ','Pathway','MP']:
    for lib in ['CC']:
        ref = enrich.dict_g2t[lib]
        tmp_pvals = enrich.get_pvalues(gl,ref,19061,gene_type='H_symbol',return_gene_info=True,max_gene_in_lib=1000)[2]
        tmp_pvals = tmp_pvals.set_index('Term_name').genelist
        ipf_enrichment = ipf_enrichment.append(tmp_pvals)

M:\Dropbox\Scripts\Python codes\IPF drug discovery\Enrichment.py:94: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  enriched_genes = ref.loc[query,report.index].apply(lambda x: geneid_table.loc[x[x==1].index,'h_symbol'].dropna().unique().tolist(),axis = 0)


In [ ]:
candidate_enrichment = enrichment_profiles.loc[top_candidates.index].dropna()
reg_vector = -1*(candidate_enrichment.reg == 'up') + 1*(candidate_enrichment.reg == 'dn')
candidate_enrichment.Raw_p_value = reg_vector*np.log10(candidate_enrichment.Raw_p_value)
cmpd_enrichment_matrix = candidate_enrichment.ix[:,:2]
cmpd_enrichment_matrix['cmpd'] = [x.split('_')[0] for x in cmpd_enrichment_matrix.index]
cmpd_enrichment_matrix = cmpd_enrichment_matrix.groupby(['cmpd','Term_name']).median()
cmpd_enrichment_matrix.reset_index(inplace = True)
cmpd_enrichment_matrix = cmpd_enrichment_matrix.pivot(index = 'cmpd',columns = 'Term_name',values = 'Raw_p_value').fillna(0)
g2t = candidate_enrichment.ix[:,[1,3]]
g2t.genelist = g2t.genelist.apply(lambda x: x.split(', '))
g2t = g2t.groupby('Term_name').sum()
g2t.genelist = g2t.genelist.apply(lambda x: ', '.join((set(x))))
g2t.transpose().append(cmpd_enrichment_matrix).to_csv('19 candidate cmpd enrichment matrix.txt',sep = '\t')

In [ ]:
import pandas as pd
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans,AgglomerativeClustering
import sklearn.preprocessing as preprocessing 
from scipy.spatial.distance import pdist,squareform

def t2g_series_parser(t2g):
    import pandas as pd
    if not isinstance(t2g, pd.Series):
        if t2g.shape[1] == 1:
            t2g = t2g.ix[:,0]
        else:
            raise ValueError('Input dataframe must be a pandas Series object!')
    if ', ' in t2g.ix[0,0]:
        sep = ', '
        print('Genelist seems to be stored as a string with separator as ", ".\nAttempt to parse...')
    elif ',' in t2g.ix[0,0]:
        sep = ','
        print('Genelist seems to be stored as a string with separator as ",".\nAttempt to parse...')
    elif ' ' in t2g.ix[0,0]:
        sep = ' '
        print('Genelist seems to be stored as a string with separator as ",".\nAttempt to parse...')
    split_attempt = t2g.apply(lambda x: x.split(sep))
    if ((split_attempt.apply(len) == 1).unique()==[True])[0]:
        raise ValueError('Input was not parsed correctly, plsease check!')
    else:
        print('Parsing is successful!')
    # use str.split to split and expand series
    t2g_matrix = t2g.str.split(sep,expand=True)
    # stack, de-stack, reset_index, and pivot it!
    t2g_matrix = t2g_matrix.stack().reset_index()
    t2g_matrix['cell_value'] = 1
    t2g_matrix = t2g_matrix.pivot(t2g_matrix.columns[0],0,values = 'cell_value').fillna(0)
    return t2g_matrix

def Find_n_clusters(t2g_matrix,max_clusters = 100):
    '''
    t2g_matrix is a dataframe of term by genes
    This function finds the best K number according to the highest silhouett_average value
    '''
#     pca = PCA(50)
#     t2g_matrix = pca.fit_transform(t2g_matrix)
#     min_max_scaler = preprocessing.MinMaxScaler()
#     t2g_matrix = min_max_scaler.fit_transform(t2g_matrix)
#     t2g_matrix = preprocessing.normalize(t2g_matrix,axis = 0)
#     tsne = TSNE(5,perplexity=40,verbose=2)
#     t2g_matrix = tsne.fit_transform(t2g_matrix)
    t2g_matrix = squareform(pdist(t2g_matrix,metric = 'correlation')) 
    Ks = range(10,max_clusters)
    km = [AgglomerativeClustering(n_clusters=i,affinity='precomputed',linkage='complete') for i in Ks]
    #     km = [AgglomerativeClustering(n_clusters=i) for i in Ks]
    labels_list = [km[i].fit_predict(t2g_matrix) for i in range(len(km))]
    # score_list = [km[i].fit_transform(t2g_matrix).min(axis = 1).sum() for i in range(len(km))]
    silhouette_avg_list = [silhouette_score(t2g_matrix, cluster_labels,metric = 'precomputed') for cluster_labels in labels_list]
    for i,score in enumerate(silhouette_avg_list):
        print('silhouette score for {} clusters: {:.2f}'.format(str(Ks[i]), score))
    best_idx = silhouette_avg_list.index(max(silhouette_avg_list))
    return Ks[best_idx],labels_list[best_idx]

def MEA(t2g_clustered):
    MEA_lib = pd.DataFrame()
    grouped = t2g_clustered.groupby('cluster')
    term2cluster = pd.Series(t2g_clustered.index,index = labels)
    gene_counts = grouped.agg('sum')
    cluster_size = grouped[t2g_clustered.columns[0]].agg('count')
    for cluster in gene_counts.index:
        # determine cluster associated genes selection threshold
    #     num_terms = cluster_size[cluster]
    #     num_iters = range(10000)
    #     list_idx = [np.random.randint(0,len(t2g_clustered),size = num_terms) for x in num_iters]
    #     permutation_matrix = np.array([t2g_clustered.iloc[x,1:].sum().values for x in list_idx])
    #     gene_threshold = np.percentile(permutation_matrix,0.999,axis = 0)

        # find gene significantly associated with term cluster
        tmp_gene_vector = gene_counts.loc[cluster]
        gene_threshold = tmp_gene_vector.mean() + 2*tmp_gene_vector.std()
        print('Gene threshold for metacluster {} of size {} is {}'.format(cluster+1,cluster_size.loc[cluster],gene_threshold))
        genes = gene_counts.columns[tmp_gene_vector>gene_threshold].tolist()
        # reconstruct tmp_gene_vector
        cluster_gene_vector = pd.Series(0,index = t2g_clustered.columns[1:])
        cluster_gene_vector[genes] = 1

        cluster_terms = term2cluster[cluster].values
        cluster_term_vector = t2g_clustered.ix[cluster_terms,1:]
        cors = cluster_term_vector.corrwith(cluster_gene_vector,axis = 1)
    #     cors = cluster_term_vector.corrwith(cluster_term_vector.mean(axis = 0),axis = 1)
        # calculating correlation only on selected genes
        cors = cluster_term_vector[genes].corrwith(cluster_term_vector[genes].mean(axis = 0),axis = 1)
        best_5_terms = cors.sort_values(ascending = False).index[:5].tolist()
        MEA_lib.ix['MEA_term_' + str(cluster+1),'genes'] = ','.join(sorted(genes))
        MEA_lib.ix['MEA_term_' + str(cluster+1),'best_terms'] = ','.join(sorted(best_5_terms))
        MEA_lib.ix['MEA_term_' + str(cluster+1),'term_size'] = len(genes)
    return MEA_lib

In [ ]:
t2g_matrix = t2g_series_parser(ipf_enrichment)
if t2g_matrix.columns[0] in enrich.geneid_table.index:
    t2g_matrix = t2g_matrix.transpose().merge(pd.DataFrame(enrich.geneid_table.ix[:,1]),left_index = True, right_index = True, how='left')
    t2g_matrix = t2g_matrix.drop_duplicates('h_entrez_id')
    t2g_matrix = t2g_matrix.set_index('h_entrez_id').transpose()
_, labels = Find_n_clusters(t2g_matrix,max_clusters=26)
t2g_matrix.insert(0,'cluster',labels)
mea_lib = MEA(t2g_matrix)
geneid_table = enrich.geneid_table
mea_lib_t2g = t2g_series_parser(mea_lib.genes).transpose()
# mea_lib_t2g = mea_lib_t2g.merge(pd.DataFrame(geneid_table.h_entrez_id),left_index=True,right_index=True,how='inner')
# mea_lib_t2g = mea_lib_t2g.drop_duplicates(subset='h_entrez_id').set_index('h_entrez_id')

# # Improved version using grouping
# # load Lincs data
# lincs_up_gl = pd.read_table('../../../Lincs_data/Lincs_Cmpd_Up_500_Genes.txt',index_col = 0)
# lincs_dn_gl = pd.read_table('../../../Lincs_data/Lincs_Cmpd_Dn_500_Genes.txt',index_col = 0)
# lincs_up_gl['reg'] = 'up'
# lincs_dn_gl['reg'] = 'dn'
# lincs_gl = lincs_up_gl.append(lincs_dn_gl)
# del lincs_up_gl
# del lincs_dn_gl
# lincs_meta_data = pd.read_table('../../../Lincs_data/Lincs_P1+P2_sig_info.txt',index_col = 0)
# all_lincs_genes = pd.read_table('../../../Lincs_data/GSE92742_Broad_LINCS_gene_info.txt').ix[:,0].tolist()

# # Running enrichment using the mea libs
# lincs_gl['cmpd_cell_dose'] = lincs_gl.pert_iname + '_' + lincs_gl.cell_id + '_' + lincs_gl.pert_idose
# lincs_gl.set_index('cmpd_cell_dose',inplace=True)
# top_candidates_gl = lincs_gl.loc[top_candidates.index]
# sea_query_gl = pd.DataFrame()
# for cmpd in top_candidates_gl.index:
#     cmpd_df = top_candidates_gl.loc[cmpd]
#     threshold = 0.25*cmpd_df.shape[0]
#     cmpd_df.Top_genes = cmpd_df.Top_genes.apply(lambda x: x.split(','))
#     all_genes = list(set(cmpd_df.Top_genes.sum()))
#     gene_counts_up = pd.Series(Counter(cmpd_df[cmpd_df.reg == 'up'].Top_genes.sum()))
#     gene_counts_dn = pd.Series(Counter(cmpd_df[cmpd_df.reg == 'dn'].Top_genes.sum()))
#     gene_counts = pd.DataFrame(0,index = all_genes,columns=['up','dn'])
#     gene_counts.ix[gene_counts_up.index,0] = gene_counts_up
#     gene_counts.ix[gene_counts_dn.index,1] = gene_counts_dn
#     gene_counts['t'] = gene_counts.up-gene_counts.dn
#     up_reg_genes = gene_counts.index[gene_counts.t>threshold].tolist()
#     dn_reg_genes = gene_counts.index[gene_counts.t<-threshold].tolist()
#     sea_query_gl.ix[cmpd.split('_')[0],'up'] = ','.join(up_reg_genes)
#     sea_query_gl.ix[cmpd.split('_')[0],'up_size'] = len(up_reg_genes)
#     sea_query_gl.ix[cmpd.split('_')[0],'dn'] = ','.join(dn_reg_genes)
#     sea_query_gl.ix[cmpd.split('_')[0],'dn_size'] = len(dn_reg_genes)
    
# mea_cmpd_sea = pd.DataFrame()
# for cmpd in sea_query_gl.index:
#     for reg in ['up','dn']:
#         gl = sea_query_gl.ix[cmpd,reg].split(',')
#         N = len(gl)
#         if [x for x in gl if x in mea_lib_t2g.index] == []:
#             print('Genelist has no common genes with ref!')
#             continue
#         tmp_pvals = enrich.get_pvalues(gl,mea_lib_t2g,19055,return_gene_info=True)[2]
#         log_tmp_pvals = pd.DataFrame((-1*(reg=='up')+(reg=='dn')) * np.log10(tmp_pvals.Raw_p_value))
#         log_tmp_pvals['cmpd'] = cmpd
#         log_tmp_pvals['genelist'] = tmp_pvals.genelist
#         mea_cmpd_sea=mea_cmpd_sea.append(log_tmp_pvals)
# # Re-enrich IPF genes against mea_lib        
# for gl, reg in zip([ipf_up,ipf_dn],['up','dn']):
#     tmp_pvals = enrich.get_pvalues(gl,mea_lib_t2g,19055, gene_type='H_symbol', return_gene_info=True)[2]
#     log_tmp_pvals = pd.DataFrame((-1*(reg=='up')+(reg=='dn')) * np.log10(tmp_pvals.Raw_p_value))
#     log_tmp_pvals['cmpd'] = 'IPF'
#     log_tmp_pvals['genelist'] = tmp_pvals.genelist
#     mea_cmpd_sea=mea_cmpd_sea.append(log_tmp_pvals)
    
# mea_cmpd_sea['abs'] = mea_cmpd_sea.Raw_p_value.abs()
# mea_cmpd_sea['term'] = mea_cmpd_sea.index
# mea_cmpd_sea = mea_cmpd_sea.sort_values(['cmpd','term','abs'],ascending=[True,True,False])
# mea_cmpd_sea = mea_cmpd_sea.groupby(['cmpd','term']).first()

# mea_cmpd_sea_matrix = mea_cmpd_sea.reset_index().ix[:,:3]
# mea_cmpd_sea_matrix = mea_cmpd_sea_matrix.pivot('cmpd','term','Raw_p_value').fillna(0).transpose()
# mea_cmpd_sea_matrix = mea_lib.merge(mea_cmpd_sea_matrix,left_index=True,right_index=True,how = 'inner')

In [ ]:
top_candidate_enrichment = pd.read_table('19 candidate cmpd enrichment matrix.txt',index_col=0)
cm_terms = [x for x in t2g_matrix.index if x in top_candidate_enrichment.columns]
top_candidate_enrichment = top_candidate_enrichment[cm_terms]
# ipf_pvals = pd.Series()
# for gl,reg in zip([ipf_up,ipf_dn],[-1,1]):
#     for lib in ['BP','Pathway','MP']:
#         ref = enrich.dict_g2t[lib]
#         tmp_pvals = enrich.get_pvalues(gl,ref,19055,gene_type='H_symbol',return_gene_info=True,max_gene_in_lib=1000)[2]
#         tmp_pvals = tmp_pvals.set_index('Term_name').Raw_p_value
#         tmp_pvals = reg*np.log10(tmp_pvals)
#         ipf_pvals = ipf_pvals.append(tmp_pvals)
# ipf_pvals = pd.DataFrame(ipf_pvals)
# ipf_pvals['term'] = ipf_pvals.index
# ipf_pvals = ipf_pvals.groupby('term').mean()
top_candidate_enrichment = top_candidate_enrichment.transpose()
top_candidate_enrichment.insert(0,'IPF_pval',ipf_pvals.loc[top_candidate_enrichment.index].values)
top_candidate_enrichment.insert(0,'cluster',t2g_matrix.ix[top_candidate_enrichment.index,0])
top_candidate_enrichment.to_clipboard()

In [ ]:
def get_quantile(apply_df):
    if apply_df.median()>=0:
        return apply_df.quantile(0.75)
    else:
        return apply_df.quantile(0.25)
    
top_candidate_enrichment = top_candidate_enrichment.drop('genelist',axis= 1)
top_candidate_enrichment.cluster = ['MEA_term_' + str(x+1) for x in top_candidate_enrichment.cluster]
top_candidate_enrichment.ix[:,1:] = top_candidate_enrichment.ix[:,1:].astype(float)
top_candidate_enrichment = top_candidate_enrichment.groupby('cluster')
top_candidate_enrichment = top_candidate_enrichment.agg(lambda x: get_quantile(x))
top_candidate_enrichment = mea_lib.merge(top_candidate_enrichment,left_index=True,right_index=True)
top_candidate_enrichment.to_clipboard()


In [ ]:
# gl = enrich.geneid_table.ix[gl,'h_entrez_id'].dropna().unique().tolist()
ref.reindex(gl).sum().loc['GO:0031226']
ref.sum().loc['GO:0031226']

In [7]:
gl = ipf_dn
gl = enrich.geneid_table.ix[gl,'h_entrez_id'].dropna().unique().tolist()
query = gl
M=19061
n = ref.sum().values
N = len(query)
X = ref.ix[query].fillna(0).sum()-1
M = np.repeat(M,ref.shape[1])
N = np.repeat(N,ref.shape[1])
pval = hypergeom.sf(X,M,n,N)
fdr = mt(pval,method='fdr_bh')[1]        
report = pd.DataFrame(fdr,index = ref.columns, columns = ['FDR_adjusted_pvalue'])
report['Raw_p_value'] = pval
report = report[report.FDR_adjusted_pvalue<=0.05]
report['Term_name'] = enrich.termid2termname.ix[report.index,0]
report.sort_values('FDR_adjusted_pvalue',inplace=True)